### IMPORT CÁC THƯ VIỆN CẦN THIẾT:
- MATPLOTLIB: DÙNG ĐỂ HIỂN THỊ DỮ LIỆU
- NUMPY: THƯ VIỆN ĐẠI SỐ TUYẾN TÍNH, DÙNG ĐỂ XỬ LÝ TOÁN HỌC VỚI MA TRẬN DỮ LIỆU
- PANDAS: DÙNG ĐỂ ĐỌC FILE VÀ THAO TÁC TRÊN FILE DỮ LIỆU


In [113]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 


### MỞ FILE SỐ LIỆU, TRỘN CÁC DÒNG VỚI NHAU MỘT CÁCH NGẪU NHIÊN

In [114]:
data = pd.read_excel("MTND2.xlsx")
data = data.drop("STT", axis = 1)
data = data.sample(frac=1)
data.head()


,GF,TBS,Natri Benzoate,Abs 260,Abs 265,Abs 270,Abs 275,Abs 280,Abs 285,Abs 290
31,2.5,3.0,2.0,0.308,0.416,0.536,0.580,0.503,0.243,0.064
32,2.5,3.0,2.5,0.340,0.446,0.565,0.603,0.519,0.251,0.073
7,2.0,2.0,1.5,0.219,0.302,0.392,0.425,0.365,0.168,0.041
4,3.0,3.0,1.0,0.263,0.386,0.519,0.577,0.503,0.234,0.050
40,1.0,2.5,0.5,0.154,0.214,0.281,0.321,0.294,0.172,0.055


### CHIA DỮ LIỆU THÀNH DỮ LIỆU HỌC (data_train) VÀ DỮ LIỆU KIỂM TRA (data_test), SAU ĐÓ CHIA THÀNH MA TRẬN ĐẦU TÍN HIỆU (Y) VÀ MA TRẬN NỒNG ĐỘ (X)

In [115]:
# Split data into train and test data
data_train = data[:int(0.7 * len(data))].values
data_test = data[int(0.7 * len(data)):].values
# Convert data table to X and Y matricses
X_train = data_train[:,0:3].astype(float) 
Y_train = data_train[:,3:].reshape(-1, 7).astype(float)
X_test = data_test[:,0:3].astype(float) 
Y_test = data_test[:,3:].reshape(-1, 7).astype(float)

In [116]:
Y_chunks = np.array_split(Y_train, len(Y_train) / 8)
X_chunks = np.array_split(X_train, len(Y_train) / 8)

### KHAI BÁO CÁC HÀM KÍCH HOẠT THƯỜNG SỬ DỤNG

In [117]:
def ReLU(input):
    return np.maximum(input, 0)
def leaky_ReLU(input):
    return np.maximum(0.01 * input, input)
def Linear(input):
    return input
def Sigmoid(input):
    return 1 / (1 + np.exp(-input))
def Swish(input):
    return input * Sigmoid(input)

### KHAI BÁO ĐẠO HÀM CỦA CÁC HOẠT KÍCH HOẠT THEO DỮ LIỆU ĐẦU VÀO

In [118]:
def ReLU_de(input):
    return (input > 0) * 1
def leaky_ReLU_de(input):
    return np.where(input > 0, 1, 0.01) 
def Linear_de(input):
    return 1
def Sigmoid_de(input):
    return np.exp(input)/ np.square(np.exp(input) + 1)
def Swish_de(input):
    return Swish(input) + Sigmoid(input)*(1 - Swish(input))

##### HÀM NHẬN VÀO MỘT MẢNG CHỨA CÁC HÀM KÍCH HOẠT Ở TỪNG LỚP, TRẢ VỀ MỘT MẢNG CÙNG ĐỘ DÀI VỚI ĐẠO HÀM TƯƠNG ỨNG CỦA CÁC HÀM ĐÓ

In [119]:
def deriative_func(func_list):
    deriative_funcs = []
    for func in func_list:
        if func.__name__ == "ReLU":
            deriative_funcs.append(ReLU_de)
        if func.__name__ == "leaky_ReLU":
            deriative_funcs.append(leaky_ReLU_de)
        if func.__name__ == "Linear":
            deriative_funcs.append(Linear_de)
        if func.__name__ == "Sigmoid":
            deriative_funcs.append(Sigmoid_de)
        if func.__name__ == "Swish":
            deriative_funcs.append(Swish_de)
    return deriative_funcs

### KHAI BÁO CLASS (MODEL) MẠNG NƠ RON NHÂN TẠO

In [120]:
class Neural_Network:
    def __init__(self, layer_nodes_list, layer_activate_funcs):
        self.layerfunc_list = layer_activate_funcs
        self.n_layers = len(layer_nodes_list)
        self.layer_deriative_funcs = deriative_func(layer_activate_funcs)
        # INIT BIAS MATRIX
        self.bias = []
        for i in range (1, self.n_layers):
            self.bias.append(0.01 * np.zeros((1, layer_nodes_list[i])))
        # INIT WEIGHTS MATRIX
        self.weights = []
        for i in range (1, self.n_layers):
            self.weights.append(0.01 * np.random.rand(layer_nodes_list[i-1], layer_nodes_list[i]))
    def fit(self, input, target, iter, learning_rate):
        for j in range (iter):
            # FORWARD PASS
            input_data = input
            layers_outputs = []
            layers_inputs = [input_data]
            for i in range (self.n_layers):
                if i > 0:
                    # print(self.weights[i - 1].shape)
                    input_data = np.dot(output_data, self.weights[i - 1]) + self.bias[i-1]
                    layers_inputs.append(input_data)
                output_data = self.layerfunc_list[i](input_data)
                layers_outputs.append(output_data)

            # BACKPROPAGATION
            delta = (layers_outputs[-1] - target)*(self.layer_deriative_funcs[-1](layers_inputs[-1]))
            d_weights = [np.dot(layers_outputs[-2].T, delta)]
            d_bias = [np.dot(np.ones((1, delta.shape[0])), delta)]

            for i in range (self.n_layers - 3, -1, -1):
                delta = np.dot(delta, self.weights[i + 1].T) * self.layer_deriative_funcs[i](layers_inputs[i + 1])
                d_weights.insert(0,np.dot(layers_outputs[i].T, delta))
                d_bias.insert(0,np.dot(np.ones((1, delta.shape[0])), delta))

            for i in range (self.n_layers - 2, -1, -1):
                self.weights[i] -= learning_rate * d_weights[i]
                self.bias[i] -= learning_rate * d_bias[i]
            # print(np.sum(np.abs(layers_outputs[-1] - target)))
    def predict(self, input):
        input_data = input
        for i in range (self.n_layers):
            if i > 0:
                input_data = np.dot(output_data, self.weights[i - 1]) + self.bias[i-1]
            output_data = self.layerfunc_list[i](input_data)
        return output_data        

In [121]:
model = Neural_Network([Y_train.shape[1], 5, 4, X_train.shape[1]], [Linear, ReLU, ReLU, Linear])
for i in range(len(Y_chunks)):
    model.fit(Y_chunks[i], X_chunks[i], iter = 1000, learning_rate=0.001)
# print(ReLU(Y_train))
# print(model.predict(Y_test))
X_pred = model.predict(Y_test)
print(X_pred)
print(X_test)

[[1.01774115 0.86908975 1.47780491]
 [2.55310933 2.40611757 1.56315638]
 [1.92781913 1.78014869 1.52839148]
 [2.3331287  2.18589819 1.55092589]
 [3.23361247 3.08735935 1.60099101]
 [1.01774115 0.86908975 1.47780491]
 [2.10349257 1.95601281 1.53815858]
 [1.01774115 0.86908975 1.47780491]
 [1.71436507 1.56646293 1.51652385]
 [1.01774115 0.86908975 1.47780491]
 [3.10344381 2.9570494  1.59375389]
 [1.01774115 0.86908975 1.47780491]
 [1.01774115 0.86908975 1.47780491]
 [2.67270789 2.52584596 1.56980582]]
[[1.  1.  0.5]
 [2.5 2.5 1. ]
 [2.5 1.  0.5]
 [2.5 1.  2.5]
 [3.  3.  3. ]
 [0.5 2.  0.5]
 [2.5 1.  1.5]
 [0.5 0.5 0.5]
 [1.2 1.5 1. ]
 [0.5 1.  0.5]
 [3.  3.  2. ]
 [0.5 1.  1. ]
 [0.5 0.5 2.5]
 [2.5 2.5 2. ]]


In [122]:
print(((X_pred - X_test) / X_test ) * 100)

[[  1.77411494 -13.09102528 195.56098188]
 [  2.12437303  -3.75529703  56.31563803]
 [-22.88723466  78.01486928 205.6782967 ]
 [ -6.67485215 118.58981902 -37.96296427]
 [  7.78708229   2.91197832 -46.63363288]
 [103.54822989 -56.54551264 195.56098188]
 [-15.86029733  95.60128077   2.54390525]
 [103.54822989  73.81794944 195.56098188]
 [ 42.86375556   4.43086172  51.65238526]
 [103.54822989 -13.09102528 195.56098188]
 [  3.44812695  -1.43168667 -20.31230529]
 [103.54822989 -13.09102528  47.78049094]
 [103.54822989  73.81794944 -40.88780362]
 [  6.90831567   1.03383832 -21.50970877]]
